##Imports

In [54]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

In [55]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yudhaislamisulistya/plants-type-datasets")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plants-type-datasets' dataset.
Path to dataset files: /kaggle/input/plants-type-datasets


In [56]:
ls /kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants

Test_Set_Folder/  Train_Set_Folder/  Validation_Set_Folder/


In [57]:
ls /kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder

 aloevera/     corn/       guava/       paddy/          soybeans/
 banana/       cucumber/   kale/        papaya/         spinach/
 bilimbi/      curcuma/    longbeans/  'peper chili'/  'sweet potatoes'/
 cantaloupe/   eggplant/   mango/       pineapple/      tobacco/
 cassava/      galangal/   melon/       pomelo/         waterapple/
 coconut/      ginger/     orange/      shallot/        watermelon/


In [58]:

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_data = datasets.ImageFolder(
    '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder',
    transform=train_transform
)
test_data = datasets.ImageFolder(
    '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Test_Set_Folder',
    transform=test_transform
)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

num_classes = len(train_data.classes)
print(f"Classes: {num_classes}")
print(f"Training images: {len(train_data)}")
print(f"Test images: {len(test_data)}")

Classes: 30
Training images: 23972
Test images: 2998


In [59]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [60]:

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(

            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),


            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),


            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),


            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes))

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



def get_mobilenet(num_classes):
    model = models.mobilenet_v2(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

In [61]:
model = get_mobilenet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("Ready to train!")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 117MB/s]


Ready to train!


In [62]:
num_epochs = 20
best_acc = 0

for epoch in range(num_epochs):
    # Train
    model.train()
    train_loss = 0
    train_correct = 0

    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_acc = 100 * train_correct / len(train_data)

    # Test
    model.eval()
    test_correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_correct += (outputs.argmax(1) == labels).sum().item()

    test_acc = 100 * test_correct / len(test_data)

    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Test Acc = {test_acc:.2f}%")

    # Save best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"  → New best! Saved model.")

print(f"\nBest Test Accuracy: {best_acc:.2f}%")

Epoch 1/20: 100%|██████████| 750/750 [04:58<00:00,  2.51it/s]


Epoch 1: Train Acc = 85.12%, Test Acc = 96.33%
  → New best! Saved model.


Epoch 2/20: 100%|██████████| 750/750 [03:32<00:00,  3.53it/s]


Epoch 2: Train Acc = 96.12%, Test Acc = 97.67%
  → New best! Saved model.


Epoch 3/20: 100%|██████████| 750/750 [03:31<00:00,  3.55it/s]


...

Epoch 18: Train Acc = 99.39%, Test Acc = 98.67%


Epoch 19/20: 100%|██████████| 750/750 [03:31<00:00,  3.54it/s]


Epoch 19: Train Acc = 99.30%, Test Acc = 98.97%
  → New best! Saved model.


Epoch 20/20: 100%|██████████| 750/750 [03:31<00:00,  3.54it/s]


Epoch 20: Train Acc = 99.14%, Test Acc = 98.47%

Best Test Accuracy: 98.97%
